In [ ]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models import TorchETM
import torch
import numpy as np

In [ ]:
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

In [ ]:
# Load model
model = TorchETM.ETM_Wrapper()

In [ ]:
model.hyperparameters['epochs'] = 5
model.hyperparameters['enc_drop'] = 0.1

In [ ]:
model.test_set(True)

In [ ]:
model.train_model(dataset, model.hyperparameters, top_words= 5)

In [ ]:
model.inference()